# NICE STATS

In [2]:
import sqlite3 as lite
import math
import time
import sys
import json
import pandas as pd
import os
import ftplib
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')

fdir = os.path.abspath(os.path.dirname('__file__'))
start_time = time.time()



/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [57]:
# Set to True to use a nice sample db
sample_data = False

# set to True for printed messages
verbose = True

In [58]:
time_format = "%Y-%m-%dT%H:%M"

if sample_data:
    database = os.path.join(fdir, '../sample_data.db')
else:
    database = os.path.join(fdir, '../data.db')



In [59]:
# Get current time rounded down to nearest 15 minutes
current_time = time.time()
current_time = current_time - (current_time % (15*60))
current_time = pd.to_datetime(current_time, unit='s')

if sample_data:
    current_time = pd.to_datetime('2016-11-21 18:30:00') 
if verbose:
    print current_time

2016-12-11 22:30:00



TODO
* Calculate rain in the last 24 hours

In [60]:
con = lite.connect(database)
cur = con.cursor()
query = """
        SELECT timestamp, rain, level, forecast 
            from {river}
        ORDER BY timestamp DESC
    """
cur.execute(query.format(river=river, limit=limit))
result = cur.fetchall()

df_all = pd.DataFrame(result, columns=['timestamp', 'cum_rain', 'level', 'forecast'])

df_all.head()

,timestamp,cum_rain,level,forecast
0,2016-12-13T03:00,NaN,NaN,0.472917
1,2016-12-13T00:00,NaN,NaN,0.770833
2,2016-12-12T21:00,NaN,NaN,0.000000
3,2016-12-12T18:00,NaN,NaN,0.000000
4,2016-12-12T15:00,NaN,NaN,0.000000


In [61]:
df_all.timestamp = pd.to_datetime(df_all.timestamp)
df_all = df_all.set_index('timestamp')
df_all = df_all.sort_index()

df_all.head(10)

,cum_rain,level,forecast
timestamp,,,
2016-06-11 21:00:00,0.000000,NaN,NaN
2016-06-11 21:15:00,0.023438,NaN,NaN
2016-06-11 21:30:00,0.037500,NaN,NaN
2016-06-11 21:45:00,0.063281,NaN,NaN
2016-06-11 22:00:00,0.091406,NaN,NaN
2016-06-11 22:15:00,0.103125,NaN,NaN
2016-06-11 22:30:00,0.105469,NaN,NaN
2016-06-11 22:45:00,0.239063,NaN,NaN
2016-06-11 23:00:00,0.309375,NaN,NaN


In [63]:
# LAST TIME THAT THE DART RAN
up_rows = df_all[df_all.level > 0.7].index
last_up = None
if len(up_rows) > 0:
    last_up = pd.to_datetime(max(up_rows))
if verbose:
    print 'current_time: ' + str(current_time)

if last_up:
    if verbose:
        print 'last_up: ' + str(last_up)

    diff = (current_time - last_up)

    last_up_days = diff.value / (24 * 60 * 60 * 1000 * 1000 * 1000)
    last_up_text = str(last_up_days) + ' DAYS'
    if last_up_days == 0:
        last_up_hours = diff.value / (60 * 60 * 1000 * 1000 * 1000)
        last_up_text = str(last_up_hours) + ' HOURS'
        if last_up_hours == 0:
            last_up_minutes = diff.value / (60 * 1000 * 1000 * 1000)
            last_up_text = str(last_up_minutes) + ' MINUTES'
    if verbose:
        print
        print 'LAST RUNNING ' + last_up_text + ' AGO'
else:
    last_up_text = "NEVER?!"
    if verbose:
        print last_up_text


current_time: 2016-12-11 22:30:00
last_up: 2016-12-11 02:15:00

LAST RUNNING 20 HOURS AGO


In [90]:
# NUMBER OF DAYS THAT THE DART HAS RUN IN DATA
df_days = df_all.resample('D').max()
#print len(df_days[df_days.level > 0.7])


# NUMBER OF DAYS THAT THE DART HAS RUN IN LAST WEEK
df_last_week = df_days[(current_time - df_days.index) < np.timedelta64(7,'D')]
days_up_week = len(df_last_week[df_last_week.level > 0.7])
max_level_week = df_last_week.level.max().round(2)

if verbose:
    print 'UP ON ' + str(days_up_week) + ' DAYS IN THE LAST 7 DAYS'
    print 'HIGHEST LEVEL IN THE LAST 7 DAYS WAS ' + str(max_level_week)

# NUMBER OF DAYS THAT DART HAS RUN IN LAST MONTH
df_last_month = df_days[(current_time - df_days.index) < np.timedelta64(30,'D')]
days_up_month = len(df_last_month[df_last_month.level > 0.7])
max_level_month = df_last_month.level.max().round(2)

if verbose:
    print 'UP ON ' + str(days_up_month) + ' DAYS IN THE LAST 30 DAYS'
    print 'HIGHEST LEVEL IN THE LAST 30 DAYS WAS ' + str(max_level_month)




UP ON 2 DAYS IN THE LAST 7 DAYS
HIGHEST LEVEL IN THE LAST 7 DAYS WAS 1.31
UP ON 7 DAYS IN THE LAST 30 DAYS
HIGHEST LEVEL IN THE LAST 30 DAYS WAS 2.29


In [91]:
# SUM OF RAIN IN THE LAST 24 HOURS ... 
# ...A bit more tricky, use cum_rain and somehow take out the max from previous day and today and sum 

# SUM OF RAIN IN THE LAST WEEK
sum_rain_week = int(round(sum(df_last_week.cum_rain.fillna(0))))

sum_rain_month = int(round(sum(df_last_month.cum_rain.fillna(0))))


# SUM OF RAIN IN THE LAST MONTH
if verbose:
    print str(sum_rain_week) + ' MM OF RAIN IN THE LAST 7 DAYS'
    print str(sum_rain_month) + ' MM OF RAIN IN THE LAST 30 DAYS'


43 MM OF RAIN IN THE LAST 7 DAYS
195 MM OF RAIN IN THE LAST 30 DAYS


In [94]:
data = {}       
data['current_time'] = current_time.value / 1000000
data['last_up_text'] = last_up_text
data['days_up_week'] = days_up_week
data['max_level_week'] = max_level_week.round(2)
data['days_up_month'] = days_up_month
data['max_level_month'] = max_level_month
data['sum_rain_week'] = sum_rain_week
data['sum_rain_month'] = sum_rain_month


filename = os.path.join(fdir, 'dart_stats.json')
with open(filename, 'w') as f:
    json.dump(data, f, indent=4)